In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv1D, AveragePooling1D, BatchNormalization, Activation,
    Dropout, Concatenate, Bidirectional, LSTM, Flatten, Dense
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

import pandas as pd
import numpy as np

In [ ]:
def build_light_pairs_conv_lstm_model(
    window=60,
    channels=1,
    filters=12,
    kernels=(3,5),
    dilations=(1,8),
    lstm_units=32,
    fc_units=(36, 18),
    conv_dropout=0.05,
    lstm_dropout=0.15,
    final_dropout=0.20,
    num_classes=3,
    lr=3e-4
):

    inp = Input(shape=(window, channels), name="spread_input")

    # ------------------------------------------------------
    # 1) PARALLEL CNN BRANCHES + MEAN POOLING
    # ------------------------------------------------------
    branch_seq_outputs = []

    for k in kernels:
        x = inp

        for d in dilations:
            # --- Conv layer 1 ---
            x = Conv1D(filters, kernel_size=k, dilation_rate=d,
                       padding="same", activation=None,
                       name=f"conv1_k{k}_d{d}")(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
            x = AveragePooling1D(pool_size=2, strides=1, padding="same")(x)
            x = Dropout(conv_dropout)(x)

            # --- Conv layer 2 ---
            x = Conv1D(filters, kernel_size=k, dilation_rate=d,
                       padding="same", activation=None,
                       name=f"conv2_k{k}_d{d}")(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
            x = AveragePooling1D(pool_size=2, strides=1, padding="same")(x)
            x = Dropout(conv_dropout)(x)

        branch_seq_outputs.append(x)

    # ------------------------------------------------------
    # 2) CONCATENATE BRANCH SEQUENCES
    # ------------------------------------------------------
    fused_seq = Concatenate(axis=-1, name="conv_fusion")(branch_seq_outputs)

    # ------------------------------------------------------
    # 3) BiLSTM OVER FUSED CONV SEQUENCES
    # ------------------------------------------------------
    lstm_out = Bidirectional(
        LSTM(lstm_units, return_sequences=False, dropout=lstm_dropout),
        name="bilstm"
    )(fused_seq)

    # ------------------------------------------------------
    # 4) RAW SPREAD SKIP → FLATTEN → CONCAT
    # ------------------------------------------------------
    raw_flat = Flatten(name="raw_flatten")(inp)

    combined = Concatenate(name="fusion_with_raw")([lstm_out, raw_flat])

    # ------------------------------------------------------
    # 5) FULLY CONNECTED HEAD
    # ------------------------------------------------------
    x = Dense(fc_units[0], activation="relu")(combined)
    x = Dropout(final_dropout)(x)

    x = Dense(fc_units[1], activation="relu")(x)
    x = Dropout(final_dropout/2)(x)

    out = Dense(num_classes, activation="softmax", name="output")(x)

    model = Model(inputs=inp, outputs=out, name="LightPairsConvBiLSTM_MeanPool")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [ ]:
model = build_light_pairs_conv_lstm_model()

model.summary()

Model: "LightPairsConvBiLSTM_MeanPool"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ spread_input        │ (None, 60, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_k3_d1         │ (None, 60, 12)    │         48 │ spread_input[0][… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_k5_d1         │ (None, 60, 12)    │         72 │ spread_input[0][… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 12)    │         48 │ conv1_k3_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 12)    │         48 │ conv1_k5_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 60, 12)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 60, 12)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d_8 │ (None, 60, 12)    │          0 │ activation_8[0][… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d_… │ (None, 60, 12)    │          0 │ activation_12[0]… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 60, 12)    │          0 │ average_pooling1… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 60, 12)    │          0 │ average_pooling1… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_k3_d1         │ (None, 60, 12)    │        444 │ dropout_10[0][0]  │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_k5_d1         │ (None, 60, 12)    │        732 │ dropout_14[0][0]  │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 12)    │         48 │ conv2_k3_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 12)    │         48 │ conv2_k5_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 60, 12)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_13       │ (None, 60, 12)    │          0 │ batch_normalizat

 Total params: 23,847 (93.15 KB)

 Trainable params: 23,655 (92.40 KB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
# ============================================================
# ===================== USER PARAMETERS =======================
# ============================================================

DL_WINDOW = 60
HOLD_DOWNSAMPLE_FRAC = 0.5

DATA_FILE = "/content/final_dl_training_dataset.csv"

# ============================================================
# ===================== LOAD & PREPROCESS DATA ===============
# ============================================================

print("Loading dataset...")
df = pd.read_csv(DATA_FILE)
df["date"] = pd.to_datetime(df["date"])

# ----- Downsample hold class -----
df_hold = df[df["label"] == 0].sample(frac=HOLD_DOWNSAMPLE_FRAC, random_state=42)
df_trade = df[df["label"] != 0]
df = pd.concat([df_hold, df_trade]).sort_values("date")
df = df.reset_index(drop=True)

print("After balancing:")
print(df["label"].value_counts())

# ----- Extract spread windows -----
spread_cols = [f"spread_{i+1}" for i in range(DL_WINDOW)]
X_raw = df[spread_cols].values
y_raw = df["label"].values

# ----- Window normalization -----
def normalize_window(w):
    mean = w.mean()
    std = w.std()
    if std == 0:
        std = 1e-8
    return (w - mean) / std

X_norm = np.array([normalize_window(w) for w in X_raw])

# ----- Reshape for Conv-LSTM: (samples, timesteps=60, channels=1) -----
X = X_norm.reshape(-1, DL_WINDOW, 1)

# ----- Convert labels → [0,1,2] -----
label_map = {-1: 0, 0: 1, +1: 2}
y = np.array([label_map[v] for v in y_raw])
y_cat = to_categorical(y, num_classes=3)

# ----- Time-based split -----
N = len(df)
train_end = int(0.7 * N)
val_end = int(0.85 * N)

X_train, y_train = X[:train_end], y_cat[:train_end]
X_val,   y_val   = X[train_end:val_end], y_cat[train_end:val_end]
X_test,  y_test  = X[val_end:], y_cat[val_end:]

print("Shapes:")
print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

# ============================================================
# ========================= TRAIN MODEL =======================
# ============================================================

checkpoint_all = ModelCheckpoint(
    filepath="/content/models_CE/model_epoch_{epoch:02d}_valLoss_{val_loss:.4f}_CE.h5",
    save_weights_only=False,
    save_freq="epoch"
)

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint_all]
)

# ============================================================
# ========================== TESTING ==========================
# ============================================================

test_loss, test_acc = model.evaluate(X_test, y_test)
print("\nTEST ACCURACY:", test_acc)

Loading dataset...
After balancing:
label
 0    34496
 1    30438
-1    30384
Name: count, dtype: int64
Shapes:
Train: (66722, 60, 1) (66722, 3)
Val:   (14298, 60, 1) (14298, 3)
Test:  (14298, 60, 1) (14298, 3)
Epoch 1/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5856 - loss: 0.8851

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accuracy: 0.5858 - loss: 0.8849 - val_accuracy: 0.7060 - val_loss: 0.7074
Epoch 2/100
1042/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6840 - loss: 0.7580

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.6840 - loss: 0.7580 - val_accuracy: 0.7144 - val_loss: 0.6922
Epoch 3/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6926 - loss: 0.7354

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.6926 - loss: 0.7354 - val_accuracy: 0.7135 - val_loss: 0.6949
Epoch 4/100
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6972 - loss: 0.7307

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 41s 25ms/step - accuracy: 0.6972 - loss: 0.7307 - val_accuracy: 0.7130 - val_loss: 0.6924
Epoch 5/100
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6976 - loss: 0.7162

...

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 41s 25ms/step - accuracy: 0.7348 - loss: 0.6139 - val_accuracy: 0.7031 - val_loss: 0.7287
Epoch 30/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7375 - loss: 0.6100

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 41s 25ms/step - accuracy: 0.7375 - loss: 0.6100 - val_accuracy: 0.7093 - val_loss: 0.7215
Epoch 31/100
1042/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7364 - loss: 0.6073

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.7364 - loss: 0.6073 - val_accuracy: 0.7076 - val_loss: 0.7374
Epoch 32/100
1042/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7411 - loss: 0.6011

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.7411 - loss: 0.6011 - val_accuracy: 0.7019 - val_loss: 0.7446
Epoch 33/100
 247/1043 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - accuracy: 0.7496 - loss: 0.5907

KeyboardInterrupt: 

Report for best model (avgpool)

In [ ]:
# ----------------------------------------------------
# LOAD ANY SAVED MODEL FILE
# ----------------------------------------------------
model_path = "/content/models_CE/model_epoch_12_valLoss_0.6797_CE.h5"   # <--- change this
model = load_model(model_path)
print("Loaded model:", model_path)

# ----------------------------------------------------
# RUN PREDICTION ON TEST SET
# ----------------------------------------------------
y_pred_probs = model.predict(X_test)     # shape: (N, 3)
y_pred = np.argmax(y_pred_probs, axis=1) # convert to class labels

y_true = np.argmax(y_test, axis=1)       # convert one-hot to labels

# ----------------------------------------------------
# CONFUSION MATRIX
# ----------------------------------------------------
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)

# ----------------------------------------------------
# CLASSIFICATION REPORT
# ----------------------------------------------------
print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))


Loaded model: /content/models_CE/model_epoch_12_valLoss_0.6797_CE.h5
447/447 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step

Confusion Matrix:
[[3791  916  152]
 [ 815 3062  775]
 [ 377 1081 3329]]

Classification Report:
              precision    recall  f1-score   support

           0     0.7608    0.7802    0.7704      4859
           1     0.6053    0.6582    0.6306      4652
           2     0.7822    0.6954    0.7363      4787

    accuracy                         0.7121     14298
   macro avg     0.7161    0.7113    0.7124     14298
weighted avg     0.7173    0.7121    0.7135     14298



Custom Loss Fn

In [ ]:
import tensorflow.keras.backend as K

In [ ]:
def weighted_categorical_crossentropy(weight_matrix):
    """
    weight_matrix: 2D matrix [true_class][predicted_class]
    """
    weight_matrix = tf.constant(weight_matrix, dtype=tf.float32)

    def loss(y_true, y_pred):
        # y_true, y_pred are (batch, num_classes)
        y_true_idx = tf.argmax(y_true, axis=-1)     # true class index
        y_pred_idx = tf.argmax(y_pred, axis=-1)     # predicted class index

        # gather weights per sample
        weights = tf.gather_nd(weight_matrix,
            tf.stack([y_true_idx, y_pred_idx], axis=1)
        )

        # standard CE
        ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

        # apply weights
        return ce * weights

    return loss

In [ ]:
penalty_matrix = [
    [1.0, 2.0, 4.0],   # true class 0 = -1
    [1.5, 1.0, 1.5],   # true class 1 =  0
    [4.0, 2.0, 1.0]    # true class 2 = +1
]

In [ ]:
def build_light_pairs_conv_lstm_model(
    window=60,
    channels=1,
    filters=12,
    kernels=(3,5),
    dilations=(1,8),
    lstm_units=32,
    fc_units=(36, 18),
    conv_dropout=0.05,
    lstm_dropout=0.15,
    final_dropout=0.20,
    num_classes=3,
    lr=3e-4
):

    inp = Input(shape=(window, channels), name="spread_input")

    # ------------------------------------------------------
    # 1) PARALLEL CNN BRANCHES + MEAN POOLING
    # ------------------------------------------------------
    branch_seq_outputs = []

    for k in kernels:
        x = inp

        for d in dilations:
            # --- Conv layer 1 ---
            x = Conv1D(filters, kernel_size=k, dilation_rate=d,
                       padding="same", activation=None,
                       name=f"conv1_k{k}_d{d}")(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
            x = AveragePooling1D(pool_size=2, strides=1, padding="same")(x)
            x = Dropout(conv_dropout)(x)

            # --- Conv layer 2 ---
            x = Conv1D(filters, kernel_size=k, dilation_rate=d,
                       padding="same", activation=None,
                       name=f"conv2_k{k}_d{d}")(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
            x = AveragePooling1D(pool_size=2, strides=1, padding="same")(x)
            x = Dropout(conv_dropout)(x)

        branch_seq_outputs.append(x)

    # ------------------------------------------------------
    # 2) CONCATENATE BRANCH SEQUENCES
    # ------------------------------------------------------
    fused_seq = Concatenate(axis=-1, name="conv_fusion")(branch_seq_outputs)

    # ------------------------------------------------------
    # 3) BiLSTM OVER FUSED CONV SEQUENCES
    # ------------------------------------------------------
    lstm_out = Bidirectional(
        LSTM(lstm_units, return_sequences=False, dropout=lstm_dropout),
        name="bilstm"
    )(fused_seq)

    # ------------------------------------------------------
    # 4) RAW SPREAD SKIP → FLATTEN → CONCAT
    # ------------------------------------------------------
    raw_flat = Flatten(name="raw_flatten")(inp)

    combined = Concatenate(name="fusion_with_raw")([lstm_out, raw_flat])

    # ------------------------------------------------------
    # 5) FULLY CONNECTED HEAD
    # ------------------------------------------------------
    x = Dense(fc_units[0], activation="relu")(combined)
    x = Dropout(final_dropout)(x)

    x = Dense(fc_units[1], activation="relu")(x)
    x = Dropout(final_dropout/2)(x)

    out = Dense(num_classes, activation="softmax", name="output")(x)

    model = Model(inputs=inp, outputs=out, name="LightPairsConvBiLSTM_MeanPool")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=weighted_categorical_crossentropy(penalty_matrix),
        metrics=["accuracy"]
    )

    return model

In [ ]:
model = build_light_pairs_conv_lstm_model()

model.summary()

Model: "LightPairsConvBiLSTM_MeanPool"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ spread_input        │ (None, 60, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_k3_d1         │ (None, 60, 12)    │         48 │ spread_input[0][… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_k5_d1         │ (None, 60, 12)    │         72 │ spread_input[0][… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 60, 12)    │         48 │ conv1_k3_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 12)    │         48 │ conv1_k5_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 60, 12)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 60, 12)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d   │ (None, 60, 12)    │          0 │ activation[0][0]  │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling1d_4 │ (None, 60, 12)    │          0 │ activation_4[0][… │
│ (AveragePooling1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 60, 12)    │          0 │ average_pooling1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 60, 12)    │          0 │ average_pooling1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_k3_d1         │ (None, 60, 12)    │        444 │ dropout[0][0]     │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_k5_d1         │ (None, 60, 12)    │        732 │ dropout_4[0][0]   │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 12)    │         48 │ conv2_k3_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 12)    │         48 │ conv2_k5_d1[0][0] │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 60, 12)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 60, 12)    │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 23,847 (93.15 KB)

 Trainable params: 23,655 (92.40 KB)

 Non-trainable params: 192 (768.00 B)

In [ ]:
# ============================================================
# ===================== USER PARAMETERS =======================
# ============================================================

DL_WINDOW = 60
HOLD_DOWNSAMPLE_FRAC = 0.5

DATA_FILE = "/content/final_dl_training_dataset.csv"

# ============================================================
# ===================== LOAD & PREPROCESS DATA ===============
# ============================================================

print("Loading dataset...")
df = pd.read_csv(DATA_FILE)
df["date"] = pd.to_datetime(df["date"])

# ----- Downsample hold class -----
df_hold = df[df["label"] == 0].sample(frac=HOLD_DOWNSAMPLE_FRAC, random_state=42)
df_trade = df[df["label"] != 0]
df = pd.concat([df_hold, df_trade]).sort_values("date")
df = df.reset_index(drop=True)

print("After balancing:")
print(df["label"].value_counts())

# ----- Extract spread windows -----
spread_cols = [f"spread_{i+1}" for i in range(DL_WINDOW)]
X_raw = df[spread_cols].values
y_raw = df["label"].values

# ----- Window normalization -----
def normalize_window(w):
    mean = w.mean()
    std = w.std()
    if std == 0:
        std = 1e-8
    return (w - mean) / std

X_norm = np.array([normalize_window(w) for w in X_raw])

# ----- Reshape for Conv-LSTM: (samples, timesteps=60, channels=1) -----
X = X_norm.reshape(-1, DL_WINDOW, 1)

# ----- Convert labels → [0,1,2] -----
label_map = {-1: 0, 0: 1, +1: 2}
y = np.array([label_map[v] for v in y_raw])
y_cat = to_categorical(y, num_classes=3)

# ----- Time-based split -----
N = len(df)
train_end = int(0.7 * N)
val_end = int(0.85 * N)

X_train, y_train = X[:train_end], y_cat[:train_end]
X_val,   y_val   = X[train_end:val_end], y_cat[train_end:val_end]
X_test,  y_test  = X[val_end:], y_cat[val_end:]

print("Shapes:")
print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

# ============================================================
# ========================= TRAIN MODEL =======================
# ============================================================

checkpoint_all = ModelCheckpoint(
    filepath="/content/models/model_epoch_{epoch:02d}_valLoss_{val_loss:.4f}.h5",
    save_weights_only=False,
    save_freq="epoch"
)

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[checkpoint_all]
)

# ============================================================
# ========================== TESTING ==========================
# ============================================================

test_loss, test_acc = model.evaluate(X_test, y_test)
print("\nTEST ACCURACY:", test_acc)

Loading dataset...
After balancing:
label
 0    34496
 1    30438
-1    30384
Name: count, dtype: int64
Shapes:
Train: (66722, 60, 1) (66722, 3)
Val:   (14298, 60, 1) (14298, 3)
Test:  (14298, 60, 1) (14298, 3)
Epoch 1/100
1043/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5659 - loss: 1.5936

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 39s 25ms/step - accuracy: 0.5659 - loss: 1.5935 - val_accuracy: 0.6952 - val_loss: 1.2032
Epoch 2/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6694 - loss: 1.2898

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 25s 24ms/step - accuracy: 0.6694 - loss: 1.2898 - val_accuracy: 0.7051 - val_loss: 1.1702
Epoch 3/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6739 - loss: 1.2713

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.6739 - loss: 1.2713 - val_accuracy: 0.7079 - val_loss: 1.1586
Epoch 4/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6817 - loss: 1.2406

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 27s 26ms/step - accuracy: 0.6817 - loss: 1.2406 - val_accuracy: 0.7054 - val_loss: 1.1733
Epoch 5/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6819 - loss: 1.2285

...

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 27s 26ms/step - accuracy: 0.6933 - loss: 1.0129 - val_accuracy: 0.6876 - val_loss: 1.3390
Epoch 42/100
1042/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6993 - loss: 1.0048

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.6993 - loss: 1.0048 - val_accuracy: 0.6984 - val_loss: 1.3353
Epoch 43/100
1042/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6925 - loss: 1.0129

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.6925 - loss: 1.0129 - val_accuracy: 0.7039 - val_loss: 1.3286
Epoch 44/100
1041/1043 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6982 - loss: 0.9949

1043/1043 ━━━━━━━━━━━━━━━━━━━━ 26s 25ms/step - accuracy: 0.6982 - loss: 0.9949 - val_accuracy: 0.7035 - val_loss: 1.3921
Epoch 45/100
 457/1043 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.7013 - loss: 0.9867

KeyboardInterrupt: 

In [ ]:
# ----------------------------------------------------
# LOAD ANY SAVED MODEL FILE
# ----------------------------------------------------
model_path = "/content/models/model_epoch_13_valLoss_1.1488.h5"   # <--- change this

model = load_model(
    model_path,
    custom_objects={"loss": weighted_categorical_crossentropy}
)

print("Loaded model:", model_path)

# ----------------------------------------------------
# RUN PREDICTION ON TEST SET
# ----------------------------------------------------
y_pred_probs = model.predict(X_test)     # shape: (N, 3)
y_pred = np.argmax(y_pred_probs, axis=1) # convert to class labels

y_true = np.argmax(y_test, axis=1)       # convert one-hot to labels

# ----------------------------------------------------
# CONFUSION MATRIX
# ----------------------------------------------------
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)

# ----------------------------------------------------
# CLASSIFICATION REPORT
# ----------------------------------------------------
print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))


Loaded model: /content/models/model_epoch_13_valLoss_1.1488.h5
447/447 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step

Confusion Matrix:
[[4006  650  203]
 [ 925 2701 1026]
 [ 500  778 3509]]

Classification Report:
              precision    recall  f1-score   support

           0     0.7376    0.8244    0.7786      4859
           1     0.6542    0.5806    0.6152      4652
           2     0.7406    0.7330    0.7368      4787

    accuracy                         0.7145     14298
   macro avg     0.7108    0.7127    0.7102     14298
weighted avg     0.7115    0.7145    0.7114     14298

